# Creating a training set using 

In [2]:
import json
import config
import openai
import tiktoken
import pandas as pd

def convert_to_jsonl(data, file_path = 'data.jsonl', chunk_size=10000):
    with open(file_path , 'w') as f:
        for i in range(0, len(data), chunk_size):
            chunk = data[i:i+chunk_size]
            for entry in chunk:
                new_entry = {}
                new_entry["prompt"] = entry["text"]+"\n\n###\n\n"
                new_entry["completion"] = " "+str(entry["entities"]) +"###"
                f.write(json.dumps(new_entry) + '\n')
                
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def read_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data


def remove_duplicates(data):
    df = pd.DataFrame(data)
    df.drop_duplicates(subset='text', keep='first', inplace=True)
    new_data = df.to_dict('records')
    return new_data



In [19]:
#'./data/humanloop_spacy_format_results.json' 
file_path = './data/spacy_data/training/data_25_05_22/dev.spacy' #'./data/ground_truth_dev_set_labels.json'  # Replace with your JSON file path
data = read_json_file(file_path)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x9c in position 1: invalid start byte

In [18]:
data[0]

{'datapoint_id': 0,
 'text': 'westleigh lodge care home, nel pan lane, leigh (wn7 5jt)',
 'entities': [[0, 25, 'building_name'],
  [27, 39, 'street_name'],
  [41, 46, 'city'],
  [48, 55, 'postcode']]}

In [13]:
data[0]['data']

'69165'

In [14]:
data_dedup = remove_duplicates(data[0]['data']) 

In [34]:
# Assuming your data is stored in a variable named 'data'
convert_to_jsonl(data_dedup, './data/open_ai_formatted.jsonl')

In [58]:
openai_api_key = config.openai_API_KEY

In [4]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def count_tokens(data):
    results = []
    for d in data:
        prompt_tokens = num_tokens_from_string(d["prompt"])
        completion_tokens = num_tokens_from_string(json.dumps(d["completion"]))
        total_tokens = prompt_tokens + completion_tokens
        results.append({
            "prompt": prompt_tokens,
            "completion": completion_tokens,
            "total_tokens": total_tokens
        })
    return pd.DataFrame(results)

In [2]:
final_file = read_jsonl('./data/open_ai_formatted.jsonl')

In [5]:
df = count_tokens(final_file)
df['ratio'] = df['completion']/df['prompt']

In [10]:
#prompt length must be 4.5 times longer than the prompt to cover all eventualitys
#I can proba
df.max(axis=0)

prompt           427.0
completion      1152.0
total_tokens    1522.0
ratio              4.6
dtype: float64

In [ ]:
openai api fine_tunes.create -t "./data/open_ai_formatted.jsonl" -m ada --suffix "address_parser"

# model ID

ft-Jb4ToW6jgZPmxXFQ4SpRO7Lt



In [63]:
openai.Model.list()

<OpenAIObject list at 0x7fef21bd9190> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sa

In [65]:
final_file[0]['prompt']

'westleigh lodge care home, nel pan lane, leigh (wn7 5jt)\n\n###\n\n'

In [66]:
final_file[0]['completion']

" [[0, 25, 'building_name'], [27, 39, 'street_name'], [41, 46, 'city'], [48, 55, 'postcode']]###"

In [71]:
openai.api_key = openai_api_key
openai.Completion.create(
    model="ada:ft-personal:address-parser-2023-05-11-17-08-44",
    prompt=final_file[0]['prompt'],
    max_tokens = 100)

<OpenAIObject text_completion id=cmpl-7F7rSTjpvq12gZ1FgNK4lZSzlf7YL at 0x7fef21bb4ef0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " [[0, 25, 'building_name'], [27, 39, 'street_name'], [41, 46, 'city'], [48, 55, 'postcode']]#################################################################################################################################################################################"
    }
  ],
  "created": 1683839494,
  "id": "cmpl-7F7rSTjpvq12gZ1FgNK4lZSzlf7YL",
  "model": "ada:ft-personal:address-parser-2023-05-11-17-08-44",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 100,
    "prompt_tokens": 24,
    "total_tokens": 124
  }
}